# Práctica 2: Sensado y análisis de video
## Detección de puntos de atención visual

### Librerias

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import mediapipe as mp
import sys
import os
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Segmentado y preparación de los Datos.

In [18]:
IMGS = "./Imagen/"
dir_list = os.listdir(IMGS)
dir_list.sort()
dfsCsv = []
source = []
for i in dir_list:
    fnames = os.listdir(IMGS + i)
    for f in fnames:
        if f.endswith(".csv"):
            dfsCsv.append(pd.read_csv(IMGS + i + f"/{f}"))
            name = f.replace('csv', 'mp4')
            source.append(IMGS + i + f"/{name}")

In [19]:
print(source[-1])
dfsCsv[-1]

./Imagen/Scarlett/SCARLETT.mp4


,filename,inicio,fin,label
0,SCARLETT.mp4,00:00,00:12,frente
1,SCARLETT.mp4,00:13,00:24,izquierdo
2,SCARLETT.mp4,00:25,00:35,central
3,SCARLETT.mp4,00:37,00:47,derecho
4,SCARLETT.mp4,00:48,01:00,palanca


In [59]:
dfsCsv[0]

,Unnamed: 0,FRENTE,RETROVISOR,IZQUIERDA,DERECHA,PALANCA
0,0,00:00 - 00:08,00:24-00:28,00:09- 00:13,00:17-00:21,00:37-00:43
1,1,00:14-00:16,0058-00:60,00:45-00:48,00:51-00:56,NaN
2,2,00:22-00:23,NaN,NaN,NaN,NaN
3,3,00:29-00:36,NaN,NaN,NaN,NaN


In [60]:
df = dfsCsv[-1]
intervalos = {}

# Iterar sobre las filas del DataFrame original
for index, row in df.iterrows():
    label = row['label']
    intervalo = row['inicio'] + '-' + row['fin']
    
    # Verificar si ya hay intervalos para esta etiqueta
    if label in intervalos:
        intervalos[label].append(intervalo)
    else:
        intervalos[label] = [intervalo]

# Crear un nuevo DataFrame a partir del diccionario de intervalos
new_df = pd.DataFrame(intervalos)

# Reordenar las columnas para que coincidan con el formato deseado
column_order = ['frente', 'izquierda', 'retrovisor', 'derecha', 'palanca']

# Convertir los nombres de las columnas a mayúsculas
new_df.columns = column_order
new_df.columns = new_df.columns.str.upper()

dfsCsv[-1] = new_df

,FRENTE,IZQUIERDA,RETROVISOR,DERECHA,PALANCA
0,00:00-00:12,00:13-00:24,00:25-00:35,00:37-00:47,00:48-01:00


### Analisis exploratorio

### Extracción de caracteristicas

### Selección de caracteristicas

### Entrenamiento de los modelos

### Implementación del modelo en tiempo real. 